# N body simulation

In [1]:
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/ngcm/training-public/master/ipython_notebook_styles/ngcmstyle.css'
HTML(url=css_file)

In [2]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16

In [Greg's original post](http://software-carpentry.org/blog/2014/10/why-we-dont-teach-testing.html) he specifically references the [Python 3 version](http://benchmarksgame.alioth.debian.org/u32/program.php?test=nbody&lang=python3&id=1) of the [n-body benchmark](http://benchmarksgame.alioth.debian.org/u32/performance.php?test=nbody). In particular, he asks how to test the `advance` function. Let's reproduce that code here.

In [12]:
# The Computer Language Benchmarks Game
# http://benchmarksgame.alioth.debian.org/
#
# originally by Kevin Carson
# modified by Tupteq, Fredrik Johansson, and Daniel Nanz
# modified by Maciej Fijalkowski
# 2to3

import sys 

def combinations(l):
    result = []
    for x in range(len(l) - 1):
        ls = l[x+1:]
        for y in ls:
            result.append((l[x],y))
    return result

PI = 3.14159265358979323
SOLAR_MASS = 4 * PI * PI
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS) }


SYSTEM = list(BODIES.values())
PAIRS = combinations(SYSTEM)


def advance(dt, n, bodies=SYSTEM, pairs=PAIRS):

    for i in range(n):
        for (([x1, y1, z1], v1, m1),
             ([x2, y2, z2], v2, m2)) in pairs:
            dx = x1 - x2
            dy = y1 - y2
            dz = z1 - z2
            mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))
            b1m = m1 * mag
            b2m = m2 * mag
            v1[0] -= dx * b2m
            v1[1] -= dy * b2m
            v1[2] -= dz * b2m
            v2[0] += dx * b1m
            v2[1] += dy * b1m
            v2[2] += dz * b1m
        for (r, [vx, vy, vz], m) in bodies:
            r[0] += dt * vx
            r[1] += dt * vy
            r[2] += dt * vz

The first thing to note is the rules of the exercise, again. We're treating the function as a black box: we believe it should be implementing an algorithm with certain behaviour, and we want to check what that behaviour is. In this case it's using the [*semi-implicit Euler method*](http://en.wikipedia.org/wiki/Semi-implicit_Euler_method) (I think: would have been nice if this were documented!) to update in time, and a [direct particle-particle method](http://en.wikipedia.org/wiki/N-body_problem#Few_bodies) to compute the spatial interactions.

That is, we can think of the algorithm as solving the ODE

$$
\begin{equation}
  \frac{d y}{dt} = F(t, y)
\end{equation}
$$

where the vector $y$ contains the positions $x$ and velocities $v$ of each body. Due to the physics of the problem (ie, there's a total energy that we want to conserve) there's an advantage in considering the positions and velocities separately, but when it comes to measure the error we don't need to do that: we just treat the `advance` function as an algorithm that takes the full vector $y_n$ as input and returns $y_{n+1}$.

What this means in practical terms is that we expect the error in time to have similar convergence rate behaviour to the standard Euler method discussed before: the convergence rate should be $1$, which can be checked both by the self-convergence being within $0.585$ and $1.585$, and also that the error in the convergence rate goes down by at least $1/3$ as the base resolution of $\Delta t$ is reduced by $2$. Checking the local truncation error would require more detailed analysis, but in principle should follow the same steps as before.

What's more interesting here is the additional checks from the complexities of the particle-particle interaction. We can use our previous techniques to check that the time update algorithm is behaving as expected. However, the particle-particle interaction itself contains many steps, and those also need testing.

Let's check the output on (a copy of) the default input, taking a *single* step.

In [13]:
import copy
bodies = copy.deepcopy(SYSTEM)
advance(dt=0.01, n=1, bodies=bodies)
print(bodies)
print(SYSTEM)

[([8.333258974782792, 4.143055187836609, -0.40343925950055554], [-1.0107743461787924, 1.8256623712304119, 0.008415761376584154], 0.011286326131968767), ([12.905197472203547, -15.102464271516935, -0.22341590526666938], [1.0827910064415354, 0.8687130181696082, -0.010832637401363636], 0.0017237240570597112), ([15.389488022173355, -25.913367620001488, 0.1789112133953304], [0.979090732243898, 0.5946989986476762, -0.034755955504078104], 0.0020336868699246304), ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 39.47841760435743), ([4.847494706394679, -1.1322001755782658, -0.10387422808772198], [0.606326392995832, 2.81198684491626, -0.02521836165988763], 0.03769367487038949)]
[([8.34336671824458, 4.124798564124305, -0.4035234171143214], [-1.0148533649892928, 1.8236404735352374, 0.00861323535682711], 0.011286326131968767), ([12.894369562139131, -15.111151401698631, -0.22330757889265573], [1.0821409847561863, 0.8694752833109706, -0.010821379117708814], 0.0017237240570597112), ([15.379697114850917, -25.91931460

First, let's create a utility function that will give the displacement of the bodies.

In [ ]:
def displacement(updates):
    return updates